# 範例重點
了解如何在 Keras 中，加入 regularization <br />
熟悉建立、訓練模型 <br />
熟悉將訓練結果視覺化並比較結果 <br />

In [1]:
import os
import keras
from keras.layers import Input, Dense

#os.environ["CUDA_VISIBLE_DEVICES"] = ""
!nvidia-smi

Using TensorFlow backend.


/bin/sh: nvidia-smi: command not found


In [2]:
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

In [3]:
def preproc_x(x, flatten = True):
    x = x / 255.0
    if flatten :
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes = 10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [4]:
train_x = preproc_x(train_x)
test_x = preproc_x(test_x)
train_y = preproc_y(train_y)
test_y = preproc_y(test_y)

In [5]:
from keras.regularizers import l1, l2, l1_l2

def build_mlp(input_shape, output_units = 10, num_neurons = [512, 256, 128], l2_ratio = 1e-4):
    
    input_layer = Input(input_shape, name = "main_input")
    
    for i , n_units in enumerate(num_neurons):
        
        if i == 0 :
            x = Dense(units = n_units, activation = 'relu', name = "hidden_layer" + str(i+1),
                             kernel_regularizer = l2(l2_ratio)) (input_layer)
        else:
            x = Dense(units = n_units, activation = 'relu', name = "hidden_layer" + str(i+1),
                             kernel_regularizer = l2(l2_ratio)) (x)
        
    output_layer = Dense(units = output_units, activation = 'softmax', name = "main_output") (x)
    
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    return model

In [6]:
#Setting hyper-parameters
lr = 1e-3
epochs = 50
batch_size = 256
momentum = 0.95
l2_exp = [1e-2, 1e-4, 1e-8, 1e-12]

In [ ]:
results = {}

for regulizer_ratio in l2_exp:
    keras.backend.clear_session()
    print(f"Experiment with regulizer : {regulizer_ratio}")
    model = build_mlp(input_shape = train_x.shape[1:], l2_ratio = regulizer_ratio)
    optimizer = keras.optimizers.Adam(lr = lr)
    model.compile( optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])
    history = model.fit(train_x, train_y, epochs = epochs, batch_size = batch_size, shuffle = True, validation_data = [test_x, test_y])
    
    name_tag = f"regulizer_{str(regulizer)}"
    results[name_tag] = {
        'train_loss' : history.history['loss'],
        'valid_loss' : history.history['val_loss'],
        'train_acc' : history.history['acc'],
        'valid_acc' : history.history['val_acc']
    }
    

In [ ]:
import matplotlib.pyplot as plt

colors = ['r','g','b','y','k','m']
plt.figure(figsize = (8,6))

for i, cond in enumerate(results.keys()):
    plt.plot(results[cond]['train_loss'], '-', label = cond + "_train", color = colors[i])
    plt.plot(results[cond]['valid_loss'], '--', label = cond + "_valid", color = colors[i])
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

for i, cond in enumerate(results.keys()):
    plt.plot(results[cond]['train_acc'], '-', label = cond + '_train', color = colors[i])
    plt.plot(results[cond]['valid_acc'], '--',label = cond + '_valid', color = colors[i])
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

# Work
請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [ ]:
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

train_x = preproc_x(train_x, False)
test_x = preproc_x(train_x, False)
train_y = preproc_y(train_y)
test_y = preproc_y(test_y)

In [8]:
from keras.layers import Conv2D, Dropout, BatchNormalization, MaxPooling2D

def build_mlp2(input_shape, output_units = 10, regularizer = None):
    
    input_layer = Input(input_shape, name = "main_input")
    
    x = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer = regularizer)(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer = regularizer)(x)
    x = BatchNormalization()(x)
    x = MaxPooling(pool_size = (2,2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer = regularizer)(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer = regularizer)(x)
    x = BatchNormalization()(x)
    x = MaxPooling(pool_size = (2,2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(units = 512, activation = 'relu', kernel_regularizer = regularizer)(x)
    x = Dense(units = 256, activation = 'relu', kernel_regularizer = regularizer)(x)
    x = Dense(units = 128, activation = 'relu', kernel_regularizer = regularizer)(x)
    x = Dropout(0.25)(x)
    
    output_layer = Dense(units = output_units, activation = 'softmax', name = "main_output")(x)
    
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    return model

In [9]:
#Setting hyper-parameters
lr = 1e-3
regs = ['l1','l1_l2']
reg_ratios = [1e-4, 1e-6, 1e-8]
epochs = 30
batch_size = 256

In [ ]:
results = {}

for reg in regs:
    for reg_ratio in reg_ratios:
        
        keras.backend.clear_session()
        print(f"{str(reg)} regularizer with {reg_ratio} regular ratio")
        cur_reg = None
        if reg == 'l1':
            cur_reg = l1(reg_ratio)
        else:
            cur_reg = l1_l2(l1 = reg_ratio, l2 = reg_ratio)
            
        model = build_mlp2(input_shape = train_x.shape[1:],regularizer = cur_reg)
        optimizer = keras.optimizers.Adam(lr = lr)
        model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])
        
        history = model. fit(train_x, train_y, epochs = epochs, batch_size = batch_size, shuffle = True, validation_data = [test_x,test_y])
        
        name_tag = f"regulizer_{str(regulizer)}"
        results[name_tag] = {
        'train_loss' : history.history['loss'],
        'valid_loss' : history.history['val_loss'],
        'train_acc' : history.history['acc'],
        'valid_acc' : history.history['val_acc']
        }

In [ ]:
colors = ['r','g','b','y','k','m']
plt.figure(figsize = (8,6))

for i, cond in enumerate(results.keys()):
    plt.plot(results[cond]['train_loss'], '-', label = cond + "_train", color = colors[i])
    plt.plot(results[cond]['valid_loss'], '--', label = cond + "_valid", color = colors[i])
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

for i, cond in enumerate(results.keys()):
    plt.plot(results[cond]['train_acc'], '-', label = cond + '_train', color = colors[i])
    plt.plot(results[cond]['valid_acc'], '--',label = cond + '_valid', color = colors[i])
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()